### Single Vehicle Motion Validation
- PC Crash simulations vs. Pycrash
- Steering / Braking maneuvers

#### enable reloading modules

In [1]:
%load_ext autoreload
%autoreload 2

### Get Current Working Directory
- should be the directory containing the notebook

In [2]:
import os
os.getcwd()

'D:\\OneDrive\\pycrash\\projects\\validation - single vehicle motion\\notebooks'

In [3]:
# set root directory
import os
path_parent = os.path.dirname(os.getcwd())
os.getcwd()

'D:\\OneDrive\\pycrash\\projects\\validation - single vehicle motion\\notebooks'

#### If the directory shown is not the notebook directory, change it manually like:
`os.chdir('/home/jmc/Documents/pycrash/projects/')`

### Import Pycrash
- Will return default values for simulations
- Pycrash is not capable of loading environmental (terrain) files (in progress)

In [18]:
import sys
sys.path.insert(0,'D:\\OneDrive\\pycrash')
import pycrash
from pycrash.project import Project, project_info, load_project
from pycrash.vehicle import Vehicle
from pycrash.kinematics import SingleMotion
from pycrash.visualization.kinematics_compare import compare_kinematics
from pycrash.visualization.tire_details import tire_details, vertical_forces
from pycrash.visualization.cg_motion_compare import cg_motion

In [5]:
import pandas as pd
import numpy as np
import math
import plotly.io as pio
pio.renderers.default = "browser"  # <- determines how plots are displayed using Plotly
from scipy import integrate
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython import get_ipython
from IPython.display import display
get_ipython().run_line_magic('matplotlib', 'qt')

### Load Project Datafile that was created when project was instantiated
- the project object contains information needed to run other parts of Pycrash
- not necessary to load project data to run simulations - used to save data

#### Get project information - used to import project object

In [6]:
project_info('validation - single vehicle motion')

This saved project contains:
Object of type "project" with name "validation - single vehicle motion"
list objects in this order for loading project: ['validation - single vehicle motion']
Example: project_name, veh1, veh2 = load_project('project_name')


#### Load project and print contents
##### load project as `proj`

In [7]:
proj = load_project('validation - single vehicle motion')
proj.show()

----------------------------------  ------------------------  -----------  ---------------  --------------------
Project                             Description               Impact Type  Simulation Type  Note
validation - single vehicle motion  planar motion simulation               SV               pc crash comparisons
----------------------------------  ------------------------  -----------  ---------------  --------------------


### Generate dataframe of validation data
- use validation steer / brake / throttle inputs
- inputs are same dt as pycrash will have to check in future

List of column names used when loading PC Crash data

In [8]:
pc_crash_column_names = ['t', 'ax', 'ay', 'az', 'phi_deg', 'rf_fy', 'lf_fy',
                         'rr_fy', 'lr_fy', 'delta_deg', 'rf_delta_deg', 'steer', 
                         'steer_rate', 'X', 'Y', 'Z', 'roll', 'pitch', 'theta_deg',
                         'Vx', 'Vy', 'Vz', 'rf_fz', 'lf_fz', 'rr_fz', 'lr_fz',
                         'rf_alpha', 'lf_alpha', 'lr_alpha', 'rr_alpha']

Load PC Crash Data

In [9]:
df = pd.read_excel(os.path.join(path_parent, 'data', 'external', '15-mph-steer-data.xlsx'),
                            na_filter = False, header = None, names = pc_crash_column_names, skiprows = 2,
                            usecols = 'A:AD')

Convert units

In [10]:
# convert velocities to fps
df.Vx = [x * 1.46667 for x in df.Vx]
df.Vy = [x * 1.46667 for x in df.Vy]
df.Vz = [x * 1.46667 for x in df.Vz]

# convert acceleration to fps/s
df.ax = [x * 32.2 for x in df.ax]
df.ay = [x * 32.2 for x in df.ay]
df.az = [x * 32.2 for x in df.az]

# convert tire forces to lb
df.lf_fy = [x * 1000 for x in df.lf_fy]
df.rf_fy = [x * 1000 for x in df.rf_fy]
df.lr_fy = [x * 1000 for x in df.lr_fy]
df.rr_fy = [x * 1000 for x in df.rr_fy]

# steer angle in radians
df['delta_rad'] = [x / 180 * math.pi for x in df.delta_deg]

# integrate velocities to get displacements
df['Dx'] = 0 + integrate.cumtrapz(list(df.Vx), list(df.t), initial=0)
df['Dy'] = 0 + integrate.cumtrapz(list(df.Vy), list(df.t), initial=0)
df.head()

,t,ax,ay,az,phi_deg,rf_fy,lf_fy,rr_fy,lr_fy,delta_deg,rf_delta_deg,steer,steer_rate,X,Y,Z,roll,pitch,theta_deg,Vx,Vy,Vz,rf_fz,lf_fz,rr_fz,lr_fz,rf_alpha,lf_alpha,lr_alpha,rr_alpha,delta_rad,Dx,Dy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,1.055634,1.055634,0.592337,0.592337,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.2,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,1.055634,1.055634,0.592337,0.592337,0.0,0.0,0.0,0.0,0.0,2.200005,0.0
2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.4,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,1.055634,1.055634,0.592337,0.592337,0.0,0.0,0.0,0.0,0.0,4.400010,0.0
3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6.6,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,1.055634,1.055634,0.592337,0.592337,0.0,0.0,0.0,0.0,0.0,6.600015,0.0
4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,8.8,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,1.055634,1.055634,0.592337,0.592337,0.0,0.0,0.0,0.0,0.0,8.800020,0.0


### Create Vehicles

In [11]:
# PC Crash vehicle specifications
vehicle_input_dict = {"year":2004,  # <- creates dictionary of vehicle data for input
"make":"Chevrolet",
"model":"Malibu",
"weight":3298,
"vin":"1G1ZU54854F135916",
"brake":0,
"steer_ratio":15.9,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"width":70 / 12,
"length":187 / 12,
"hcg":21.5 / 12,
"lcgf":38.1 / 12,
"lcgr":67.9 / 12,
"wb":106 / 12,
"track":60 / 12,
"f_hang":38 / 12,
"r_hang":43 / 12,
"tire_d":26.2 / 12,
"tire_w":8.5 / 12,
"izz":2040,
"fwd":1,
"rwd":0,
"awd":0,
"A":100,
"B":41,
"k":1000,
"L":0,
"c":0,
"vx_initial":0,
"vy_initial":0,
"omega_z":0}

create driver input dataframe from PC Crash input data

In [12]:
end_time = df.t.max()
t = np.arange(0, end_time + 0.1, 0.1).tolist()
throttle = [0] * len(t)                      
brake = [0] * len(t)                         
steer = list(df.steer)                                   
driver_input_dict = {'t':t, 'throttle':throttle, 'brake':brake, 'steer':steer}
driver_input_df = pd.DataFrame.from_dict(driver_input_dict)

#### Assign validation data to `veh2`

In [13]:
veh2 = Vehicle('Veh2', vehicle_input_dict)
veh2.driver_input = driver_input_df
# apply validation data as a model result
veh2.model = df

Vehicle inputs for Veh2 applied succesfully


#### vehicle:
- "Vehicle" stores information about a single vehicle - all possible inputs do not need to be entered
- creating a Vehicle requires a "name" which is used to identify the vehicle in outputs / plots etc.


#### Create Pycrash vehicle
- set initial velocity
- try varying cg height - `hcg`
- you will get a `UserWarning` but it's fine

In [14]:
veh1 = Vehicle('Veh1', vehicle_input_dict)
t = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
brake = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
throttle = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
steer = [0, 0, -360, -360, -360, -360, -360, -360, -360, -360, -360, -360]

# ".time_inputs()" is an internal function that interpolates time inputs
veh1.time_inputs(t, throttle, brake, steer)
veh1.vx_initial = 15
veh1.hcg = 2   # vary cg height

Vehicle inputs for Veh1 applied succesfully
Driver inputs applied to Veh1


d:\projects\venv\lib\site-packages\pandas\core\reshape\merge.py:1113: UserWarning:

You are merging on int and float columns where the float values are not equal to their int representation



### Create Single Motion Instance
- simulation is run when instance is created

In [15]:
simulation_name = '15_mph_steer'
run = SingleMotion(simulation_name, veh1)

Maximum allowable friction: 0.76
Time step for vehicle motion (s) : 0.01
Maximum tire slip angle (deg): 10.00
Driver input for Veh1 of shape = (1101, 4)
Vehicle motion will be simulated for 11.0 seconds
hello from position data


### Plot Motion

In [21]:
run.plot_model()

### Plot Vehicle Motion

In [22]:
i = len(run.veh.model) - 1 # draw motion at end of simulation
print(f"Time: {run.veh.model.t[i]}")
run.global_motion(i)

Time: 11.0
 dx > dy -> adj_x = 1.6058392071376115, adj_y = 1
dx_min = -24, dx_max = 111
dy_min = -58.848326938482785, dy_max = 16.97398305373629


### Compare to PC Crash Model

In [16]:
# calculate vehicle slip angle for pycrash model - need to correct
phi_rad = []
phi_deg = []
for i in range(len(run.veh.model.t)):
    phi_rad.append(math.atan2(run.veh.model.vy[i], run.veh.model.vx[i]) * - 1) 
    phi_deg.append(math.atan2(run.veh.model.vy[i], run.veh.model.vx[i]) * -1 *(180 / math.pi))
    
run.veh.model['phi_rad'] = phi_rad
run.veh.model['phi_deg'] = phi_deg

In [19]:
compare_kinematics(run.veh.model, df, 'pycrash', 'validate')

### Get position data for validation vehicle

In [20]:
cg_motion(run.veh.model, df, 'pycrash', 'validate')

 dx > dy -> adj_x = 1.5102127125020537, adj_y = 1
dx_min = 0, dx_max = 81
dy_min = -43.848326938482785, dy_max = 1.973983053736289


### Tire Details
- Pycrash model only

In [21]:
tire_details(run.veh)

In [22]:
vertical_forces(run.veh)

In [36]:
vertical_forces(run.veh)

In [37]:
vertical_forces(run.veh)